In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

# 1 - Load Fashion MNIST 

In [2]:
# put the original *.gz files under 'your ipython notework location/data/fashion/'
# Otherwise it'll download MNIST instead!
fashion_mnist = input_data.read_data_sets('data/fashion', one_hot=True)

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


# 2 - Convolutional Neural Networks 

In [8]:
#Use He initialization 
def weight_initialization(shape):
    if len(shape) == 4:
        stride = shape[0]
        n_input = shape[2]
        sd = math.sqrt(2.0/ (n_input * stride * stride))
        
    elif len(shape) == 2:
        n_in = shape[0] 
        sd = math.sqrt(2.0/ n_in)
        
    initial = tf.truncated_normal(shape, stddev=sd)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [4]:
def create_placeholders():
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    keep_prob = tf.placeholder(tf.float32)
    return x, y_, keep_prob
    
def forward_cost(x, y_, keep_prob):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    #conv then pool
    W_conv1 = weight_initialization([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    #conv then pool
    W_conv2 = weight_initialization([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    #regular layer with dropout
    W_fc1 = weight_initialization([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    #regular layer
    W_fc2 = weight_initialization([1024, 10])
    b_fc2 = bias_variable([10])
    ZL = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    #compute cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=ZL))
    return ZL, cost


In [15]:
x, y_, keep_prob = create_placeholders()
ZL, cost = forward_cost(x, y_, keep_prob)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
correct_prediction = tf.equal(tf.argmax(ZL,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch = fashion_mnist.train.next_batch(50)
        optimizer.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.8})

        if i % 1000 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %.3f' % (i, train_accuracy))
        

    print('test accuracy %.3f' % accuracy.eval(feed_dict={x: fashion_mnist.test.images, y_: fashion_mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.160
step 1000, training accuracy 0.920
step 2000, training accuracy 1.000
step 3000, training accuracy 0.980
step 4000, training accuracy 0.960
step 5000, training accuracy 0.920
step 6000, training accuracy 0.920
step 7000, training accuracy 0.960
step 8000, training accuracy 0.980
step 9000, training accuracy 0.980
test accuracy 0.917


## It achieves 91.7% test accuaracy quickly! Next step, ResNet?